<a href="https://colab.research.google.com/github/Ph1so/Disease-Prediction/blob/main/disease_prediction_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report

from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn import tree

%matplotlib inline

# Import and understand data

In [3]:
data = pd.read_csv('/content/disease prediction data set.csv')

In [4]:
data.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data1 = pd.read_csv('/content/Symptom-severity.csv')
data1.head()

,Symptom,weight
0,itching,1
1,skin_rash,3
2,nodal_skin_eruptions,4
3,continuous_sneezing,4
4,shivering,5


In [6]:
sym = np.array(data1['Symptom'].unique())
sym.shape

(132,)

In [7]:
cols = data.columns
df = data[cols].values.flatten()
print(f"Shape: {df.shape}")
df

Shape: (88560,)


array(['Fungal infection', 'itching', ' skin_rash', ..., nan, nan, nan],
      dtype=object)

In [8]:
series_data = pd.Series(df)
series_data = series_data.str.strip()
series_data = series_data.values.reshape(data.shape)

data = pd.DataFrame(series_data, columns=data.columns)

data = data.fillna(0)
data.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Fungal infection,itching,skin_rash,dischromic _patches,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
vals = data.values
symptoms = data1['Symptom'].unique()

for i in range(len(symptoms)):
    vals[vals == symptoms[i]] = data1[data1['Symptom'] == symptoms[i]]['weight'].values[0]

d = pd.DataFrame(vals, columns=cols)

d = d.replace('dischromic _patches', 0)
d = d.replace('spotting_ urination',0)
data = d.replace('foul_smell_of urine',0)
data.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,1,3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Fungal infection,3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Fungal infection,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Fungal infection,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Fungal infection,1,3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
(data[cols] == 0).all()

data['Disease'].value_counts()

data['Disease'].unique()

df = data.iloc[:,1:].values
labels = data['Disease'].values

# Training

In [38]:
X_train, X_test, y_train, y_test = train_test_split(df, labels, shuffle=True, test_size = 0.33)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(3296, 17) (1624, 17) (3296,) (1624,)


Logistic Regression

In [39]:
lr_model = LogisticRegression(solver='saga', max_iter=2500)
lr_model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(max_iter=2500, solver='saga')

Support Vector Machine

In [40]:
svm_model = svm.SVC()
svm_model.fit(X_train, y_train)

SVC()

Decision Tree Classifier

In [41]:
dtc_model = tree.DecisionTreeClassifier()
dtc_model.fit(X_train, y_train)

DecisionTreeClassifier()

# Evaluation

In [25]:
def test(model, X_test):
  y = model.predict(X_test)
  print(f"Prediction: {y}")

In [26]:
def report(model, X_test, y_test):
    y_pred = model.predict(X_test)

    report = classification_report(y_test, y_pred)

    acc = accuracy_score(y_test, y_pred)

    print(f"Accuracy : {acc*100:.4f} %\n")
    print("Classification report: \n")
    print(report)

Logistic Regression

In [27]:
report(lr_model, X_test, y_test)

Accuracy : 91.0569 %

Classification report: 

                                         precision    recall  f1-score   support

(vertigo) Paroymsal  Positional Vertigo       0.93      0.82      0.87        17
                                   AIDS       0.85      0.97      0.90        29
                                   Acne       1.00      1.00      1.00        25
                    Alcoholic hepatitis       1.00      0.75      0.86        20
                                Allergy       0.66      0.88      0.75        24
                              Arthritis       0.77      0.96      0.86        25
                       Bronchial Asthma       0.71      0.55      0.62        22
                   Cervical spondylosis       0.75      0.52      0.62        23
                            Chicken pox       1.00      1.00      1.00        25
                    Chronic cholestasis       0.93      0.93      0.93        27
                            Common Cold       1.00      1.00 

Support Vector Machine

In [28]:
report(svm_model, X_test, y_test)

Accuracy : 92.4797 %

Classification report: 

                                         precision    recall  f1-score   support

(vertigo) Paroymsal  Positional Vertigo       0.88      0.82      0.85        17
                                   AIDS       0.97      1.00      0.98        29
                                   Acne       1.00      1.00      1.00        25
                    Alcoholic hepatitis       0.94      0.80      0.86        20
                                Allergy       0.81      0.88      0.84        24
                              Arthritis       0.86      0.96      0.91        25
                       Bronchial Asthma       0.92      0.55      0.69        22
                   Cervical spondylosis       0.84      0.70      0.76        23
                            Chicken pox       1.00      1.00      1.00        25
                    Chronic cholestasis       0.83      0.93      0.88        27
                            Common Cold       1.00      1.00 

Decision Tree Classifier

In [37]:
report(dtc_model, X_test, y_test)

Accuracy : 99.3842 %

Classification report: 

                                         precision    recall  f1-score   support

(vertigo) Paroymsal  Positional Vertigo       1.00      1.00      1.00        39
                                   AIDS       1.00      1.00      1.00        34
                                   Acne       1.00      1.00      1.00        41
                    Alcoholic hepatitis       1.00      1.00      1.00        38
                                Allergy       0.88      1.00      0.94        45
                              Arthritis       1.00      1.00      1.00        48
                       Bronchial Asthma       1.00      1.00      1.00        36
                   Cervical spondylosis       0.93      1.00      0.96        40
                            Chicken pox       1.00      1.00      1.00        41
                    Chronic cholestasis       1.00      1.00      1.00        36
                            Common Cold       1.00      1.00 

# Save model

In [ ]:
import joblib

# Save the trained model to a file
joblib.dump(lr_model, 'linear_regression_model.pkl')